# Lab | Multi-agent bidding

# Multi-agent decentralized speaker selection

This notebook showcases how to implement a multi-agent simulation without a fixed schedule for who speaks when. Instead the agents decide for themselves who speaks. We can implement this by having each agent bid to speak. Whichever agent's bid is the highest gets to speak.

We will show how to do this in the example below that showcases a fictitious presidential debate.

## Import LangChain related modules

In [26]:
!pip install langchain-openai
#pip install -U "langchain-core[openai]" "langchain-community" langchain-openai langgraph langsmith

ERROR: Operation cancelled by user
^C


In [1]:
from typing import Callable, List, Dict, Any
import tenacity
import re
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.messages import (
    HumanMessage,
    SystemMessage,
)
from langchain_openai import ChatOpenAI

In [8]:
import os
from google.colab import userdata

# 1) Prefer Colab Secrets (sidebar ▸ Secrets ▸ add OPENAI_API_KEY)
api_key = userdata.get("OPENAI_API_KEY")

# 2) Fallback: prompt securely if not found in Secrets
if not api_key:
    from getpass import getpass
    api_key = getpass("Enter OPENAI_API_KEY (input hidden): ")

# 3) Set env var used by openai + langchain_openai
os.environ["OPENAI_API_KEY"] = api_key.strip()

#  define the Python variable the notebook expects
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]

# Sanity check (does not reveal the key)
print("OPENAI_API_KEY set:", bool(os.environ.get("OPENAI_API_KEY")))
print("Python var set:", bool(OPENAI_API_KEY))

OPENAI_API_KEY set: True
Python var set: True


## `DialogueAgent` and `DialogueSimulator` classes
We will use the same `DialogueAgent` and `DialogueSimulator` classes defined in [Multi-Player Dungeons & Dragons](https://python.langchain.com/en/latest/use_cases/agent_simulations/multi_player_dnd.html).

In [9]:
class DialogueAgent:
    def __init__(
        self,
        name: str,
        system_message: SystemMessage,
        model: ChatOpenAI,
    ) -> None:
        self.name = name
        self.system_message = system_message
        self.model = model
        self.prefix = f"{self.name}: "
        self.reset()

    def reset(self):
        self.message_history = ["Here is the conversation so far."]

    def send(self) -> str:
        """
        Applies the chatmodel to the message history
        and returns the message string
        """
        message = self.model.invoke(
            [
                self.system_message,
                HumanMessage(content="\n".join(self.message_history + [self.prefix])),
            ]
        )
        return message.content

    def receive(self, name: str, message: str) -> None:
        """
        Concatenates {message} spoken by {name} into message history
        """
        self.message_history.append(f"{name}: {message}")


class DialogueSimulator:
    def __init__(
        self,
        agents: List[DialogueAgent],
        selection_function: Callable[[int, List[DialogueAgent]], int],
    ) -> None:
        self.agents = agents
        self._step = 0
        self.select_next_speaker = selection_function

    def reset(self):
        for agent in self.agents:
            agent.reset()

    def inject(self, name: str, message: str):
        """
        Initiates the conversation with a {message} from {name}
        """
        for agent in self.agents:
            agent.receive(name, message)

        # increment time
        self._step += 1

    def step(self) -> tuple[str, str]:
        # 1. choose the next speaker
        speaker_idx = self.select_next_speaker(self._step, self.agents)
        speaker = self.agents[speaker_idx]

        # 2. next speaker sends message
        message = speaker.send()

        # 3. everyone receives message
        for receiver in self.agents:
            receiver.receive(speaker.name, message)

        # 4. increment time
        self._step += 1

        return speaker.name, message

## `BiddingDialogueAgent` class
We define a subclass of `DialogueAgent` that has a `bid()` method that produces a bid given the message history and the most recent message.

In [10]:
class BiddingDialogueAgent(DialogueAgent):
    def __init__(
        self,
        name,
        system_message: SystemMessage,
        bidding_template: PromptTemplate,
        model: ChatOpenAI,
    ) -> None:
        super().__init__(name, system_message, model)
        self.bidding_template = bidding_template

    def bid(self) -> str:
        """
        Asks the chat model to output a bid to speak
        """
        prompt = PromptTemplate(
            input_variables=["message_history", "recent_message"],
            template=self.bidding_template,
        ).format(
            message_history="\n".join(self.message_history),
            recent_message=self.message_history[-1],
        )
        bid_string = self.model.invoke([SystemMessage(content=prompt)]).content
        return bid_string

## Define participants and debate topic

In [11]:
character_names = ["Donald Trump", "Kanye West", "Elizabeth Warren"]
topic = "transcontinental high speed rail"
word_limit = 50

## Generate system messages

In [12]:
game_description = f"""Here is the topic for the presidential debate: {topic}.
The presidential candidates are: {', '.join(character_names)}."""

player_descriptor_system_message = SystemMessage(
    content="You can add detail to the description of each presidential candidate."
)

def generate_character_description(character_name):
    character_specifier_prompt = [
        player_descriptor_system_message,
        HumanMessage(
            content=f"""{game_description}
            Please reply with a creative description of the presidential candidate, {character_name}, in {word_limit} words or less, that emphasizes their personalities.
            Speak directly to {character_name}.
            Do not add anything else."""
        ),
    ]
    # v1 FIX: Use .invoke() instead of callable syntax
    character_description = ChatOpenAI(temperature=1.0, openai_api_key=OPENAI_API_KEY).invoke(
        character_specifier_prompt
    ).content
    return character_description

def generate_character_header(character_name, character_description):
    return f"""{game_description}
Your name is {character_name}.
You are a presidential candidate.
Your description is as follows: {character_description}
You are debating the topic: {topic}.
Your goal is to be as creative as possible and make the voters think you are the best candidate.
"""

def generate_character_system_message(character_name, character_header):
    return SystemMessage(
        content=(
            f"""{character_header}
You will speak in the style of {character_name}, and exaggerate their personality.
You will come up with creative ideas related to {topic}.
Do not say the same things over and over again.
Speak in the first person from the perspective of {character_name}
For describing your own body movements, wrap your description in '*'.
Do not change roles!
Do not speak from the perspective of anyone else.
Speak only from the perspective of {character_name}.
Stop speaking the moment you finish speaking from your perspective.
Never forget to keep your response to {word_limit} words!
Do not add anything else."""
        )
    )

# Generate all character data
character_descriptions = [
    generate_character_description(character_name) for character_name in character_names
]
character_headers = [
    generate_character_header(character_name, character_description)
    for character_name, character_description in zip(
        character_names, character_descriptions
    )
]
character_system_messages = [
    generate_character_system_message(character_name, character_header)
    for character_name, character_header in zip(character_names, character_headers)
]


In [13]:
for (
    character_name,
    character_description,
    character_header,
    character_system_message,
) in zip(
    character_names,
    character_descriptions,
    character_headers,
    character_system_messages,
):
    print(f"\n\n{character_name} Description:")
    print(f"\n{character_description}")
    print(f"\n{character_header}")
    print(f"\n{character_system_message.content}")



Donald Trump Description:

Donald Trump: Brash and controversial, you are known for your bold and aggressive style of leadership. Your determination and confidence have fueled your political career, drawing both fervent supporters and vehement critics. Your outspoken nature and unpredictability continue to shape the political landscape.

Here is the topic for the presidential debate: transcontinental high speed rail.
The presidential candidates are: Donald Trump, Kanye West, Elizabeth Warren.
Your name is Donald Trump.
You are a presidential candidate.
Your description is as follows: Donald Trump: Brash and controversial, you are known for your bold and aggressive style of leadership. Your determination and confidence have fueled your political career, drawing both fervent supporters and vehement critics. Your outspoken nature and unpredictability continue to shape the political landscape.
You are debating the topic: transcontinental high speed rail.
Your goal is to be as creative as

## Output parser for bids
We ask the agents to output a bid to speak. But since the agents are LLMs that output strings, we need to
1. define a format they will produce their outputs in
2. parse their outputs

We can subclass the [RegexParser](https://github.com/langchain-ai/langchain/blob/master/langchain/output_parsers/regex.py) to implement our own custom output parser for bids.

In [14]:
from langchain_core.output_parsers import BaseOutputParser
import re
from typing import Dict, Any

class BidOutputParser(BaseOutputParser):
    """LangChain v1 replacement for RegexParser."""

    def get_format_instructions(self) -> str:
        return "Your response should be an integer delimited by angled brackets, like this: <int>."

    def parse(self, text: str) -> Dict[str, Any]:
        # Extract bid from <123> format
        match = re.search(r"<(\d+)>", text)
        bid = int(match.group(1)) if match else 0
        return {"bid": bid}

    @property
    def _type(self) -> str:
        return "bid_parser"

# Usage - same as before
bid_parser = BidOutputParser()

## Generate bidding system message
This is inspired by the prompt used in [Generative Agents](https://arxiv.org/pdf/2304.03442.pdf) for using an LLM to determine the importance of memories. This will use the formatting instructions from our `BidOutputParser`.

In [15]:
def generate_character_bidding_template(character_header):
    bidding_template = f"""{character_header}

```
{{message_history}}
```

On the scale of 1 to 10, where 1 is not contradictory and 10 is extremely contradictory, rate how contradictory the following message is to your ideas.

```
{{recent_message}}
```

{bid_parser.get_format_instructions()}
Do nothing else.
    """
    return bidding_template


character_bidding_templates = [
    generate_character_bidding_template(character_header)
    for character_header in character_headers
]

In [16]:
for character_name, bidding_template in zip(
    character_names, character_bidding_templates
):
    print(f"{character_name} Bidding Template:")
    print(bidding_template)

Donald Trump Bidding Template:
Here is the topic for the presidential debate: transcontinental high speed rail.
The presidential candidates are: Donald Trump, Kanye West, Elizabeth Warren.
Your name is Donald Trump.
You are a presidential candidate.
Your description is as follows: Donald Trump: Brash and controversial, you are known for your bold and aggressive style of leadership. Your determination and confidence have fueled your political career, drawing both fervent supporters and vehement critics. Your outspoken nature and unpredictability continue to shape the political landscape.
You are debating the topic: transcontinental high speed rail.
Your goal is to be as creative as possible and make the voters think you are the best candidate.


```
{message_history}
```

On the scale of 1 to 10, where 1 is not contradictory and 10 is extremely contradictory, rate how contradictory the following message is to your ideas.

```
{recent_message}
```

Your response should be an integer deli

## Use an LLM to create an elaborate on debate topic

In [17]:
topic_specifier_prompt = [
    SystemMessage(content="You can make a task more specific."),
    HumanMessage(
        content=f"""{game_description}

        You are the debate moderator.
        Please make the debate topic more specific.
        Frame the debate topic as a problem to be solved.
        Be creative and imaginative.
        Please reply with the specified topic in {word_limit} words or less.
        Speak directly to the presidential candidates: {', '.join(character_names)}.  # Fixed unpacking
        Do not add anything else."""
    ),
]

# v1 FIX: Use .invoke()
specified_topic = ChatOpenAI(temperature=1.0).invoke(
    topic_specifier_prompt
).content

print(f"Original topic:\n{topic}\n")
print(f"Detailed topic:\n{specified_topic}\n")

Original topic:
transcontinental high speed rail

Detailed topic:
Candidates, the debate topic is: "Design a plan for a futuristic, sustainable, and equitable transcontinental high-speed rail system powered by renewable energy that connects all major cities in the U.S. within 3 hours. Address funding, technology, environmental impact, and social equity in your proposals."



## Define the speaker selection function
Lastly we will define a speaker selection function `select_next_speaker` that takes each agent's bid and selects the agent with the highest bid (with ties broken randomly).

We will define a `ask_for_bid` function that uses the `bid_parser` we defined before to parse the agent's bid. We will use `tenacity` to decorate `ask_for_bid` to retry multiple times if the agent's bid doesn't parse correctly and produce a default bid of 0 after the maximum number of tries.

In [18]:
@tenacity.retry(
    stop=tenacity.stop_after_attempt(2),
    wait=tenacity.wait_none(),  # No waiting time between retries
    retry=tenacity.retry_if_exception_type(ValueError),
    before_sleep=lambda retry_state: print(
        f"ValueError occurred: {retry_state.outcome.exception()}, retrying..."
    ),
    retry_error_callback=lambda retry_state: 0,
)  # Default value when all retries are exhausted
def ask_for_bid(agent) -> str:
    """
    Ask for agent bid and parses the bid into the correct format.
    """
    bid_string = agent.bid()
    bid = int(bid_parser.parse(bid_string)["bid"])
    return bid

In [19]:
import numpy as np


def select_next_speaker(step: int, agents: List[DialogueAgent]) -> int:
    bids = []
    for agent in agents:
        bid = ask_for_bid(agent)
        bids.append(bid)

    # randomly select among multiple agents with the same bid
    max_value = np.max(bids)
    max_indices = np.where(bids == max_value)[0]
    idx = np.random.choice(max_indices)

    print("Bids:")
    for i, (bid, agent) in enumerate(zip(bids, agents)):
        print(f"\t{agent.name} bid: {bid}")
        if i == idx:
            selected_name = agent.name
    print(f"Selected: {selected_name}")
    print("\n")
    return idx

## Main Loop

In [20]:
characters = []
for character_name, character_system_message, bidding_template in zip(
    character_names, character_system_messages, character_bidding_templates
):
    characters.append(
        BiddingDialogueAgent(
            name=character_name,
            system_message=character_system_message,
            model=ChatOpenAI(temperature=0.2),
            bidding_template=bidding_template,
        )
    )

In [21]:
max_iters = 10
n = 0

simulator = DialogueSimulator(agents=characters, selection_function=select_next_speaker)
simulator.reset()
simulator.inject("Debate Moderator", specified_topic)
print(f"(Debate Moderator): {specified_topic}")
print("\n")

while n < max_iters:
    name, message = simulator.step()
    print(f"({name}): {message}")
    print("\n")
    n += 1

(Debate Moderator): Candidates, the debate topic is: "Design a plan for a futuristic, sustainable, and equitable transcontinental high-speed rail system powered by renewable energy that connects all major cities in the U.S. within 3 hours. Address funding, technology, environmental impact, and social equity in your proposals."


Bids:
	Donald Trump bid: 8
	Kanye West bid: 1
	Elizabeth Warren bid: 1
Selected: Donald Trump


(Donald Trump): I will build the greatest high-speed rail system the world has ever seen! It will be gold-plated, luxurious, and run on the most tremendous renewable energy sources. I'll make sure every American can ride it in style and comfort. It will be huge!


Bids:
	Donald Trump bid: 2
	Kanye West bid: 8
	Elizabeth Warren bid: 9
Selected: Elizabeth Warren


(Elizabeth Warren): I envision a high-speed rail system that is not only efficient and sustainable but also accessible to all Americans. By investing in renewable energy sources and innovative technology, we 



---
 ## **EXERCISE EXPERIMENTS**


In [22]:
# ============================================================
# Goal: Relevance-based bidding (instead of contradiction-based)
# ============================================================

from collections import Counter
import numpy as np
from langchain_core.messages import SystemMessage
from langchain_openai import ChatOpenAI

# --- Safety checks (keeps this cell self-contained but minimal-risk) ---
required = ["BiddingDialogueAgent", "DialogueSimulator", "ask_for_bid", "bid_parser"]
missing = [name for name in required if name not in globals()]
if missing:
    raise NameError(
        f"Missing required definitions from earlier lab cells: {missing}. "
        "Run the core lab cells first (classes + bid_parser + ask_for_bid)."
    )

# -----------------------------
#  Define a NEW mini-scenario (AI engineer flavored)
# -----------------------------
exercise_character_names = ["Ingestion Engineer", "Retrieval Engineer", "Evaluation Engineer"]

exercise_character_system_messages = [
    SystemMessage(
        content=(
            "You are the Ingestion Engineer. You focus on data sourcing, cleaning, chunking, metadata, "
            "and building an ingestion pipeline for a RAG chatbot. Keep replies practical and concrete."
        )
    ),
    SystemMessage(
        content=(
            "You are the Retrieval Engineer. You focus on embeddings, FAISS index choices, retrieval strategy, "
            "filters/metadata, reranking, and prompt grounding. Keep replies practical and concrete."
        )
    ),
    SystemMessage(
        content=(
            "You are the Evaluation Engineer. You focus on test sets, metrics, failure modes (hallucination), "
            "human evaluation, and demo design. Keep replies practical and concrete."
        )
    ),
]

exercise_topic = (
    "Team: design a Pride & Prejudice Q&A chatbot using a RAG approach. "
    "We need: ingestion plan, vector store setup, prompt templates, and an evaluation plan with demo questions."
)

# -----------------------------
# Exercise change: RELEVANCE-BASED bidding template
#    (Plus one important guardrail: if you were the last speaker, bid low)
# -----------------------------
def generate_relevance_bidding_template(role_header: str) -> str:
    return f"""{role_header}

Conversation so far: {{message_history}}

Decide whether YOU should speak next.

Rate from 1 to 10 how important it is for you (given your role) to respond to the most recent message
in order to make progress toward the goal. Higher = more important.

Rules:
- If the most recent message was spoken by you, your bid MUST be <1>.
- If you have nothing new to add (would repeat), bid <1> or <2>.
- If the recent message asks a question in your domain, bid <8> to <10>.
- If you can add a concrete next step / artifact, bid higher.

Most recent message: {{recent_message}}

{bid_parser.get_format_instructions()}

Do nothing else.
"""

# We'll store a "role header" inside the template itself (simple + consistent with lab style)
exercise_role_headers = [
    "Role: Ingestion Engineer (data sources, cleaning, chunking, metadata).",
    "Role: Retrieval Engineer (embeddings, FAISS setup, retrieval strategy, grounding).",
    "Role: Evaluation Engineer (metrics, eval set, demos, failure analysis).",
]

exercise_bidding_templates = [
    generate_relevance_bidding_template(h) for h in exercise_role_headers
]

# -----------------------------
#  Build agents (re-using BiddingDialogueAgent)
# -----------------------------
exercise_agents = []
for name, sys_msg, bid_tmpl in zip(
    exercise_character_names, exercise_character_system_messages, exercise_bidding_templates
):
    exercise_agents.append(
        BiddingDialogueAgent(
            name=name,
            system_message=sys_msg,
            model=ChatOpenAI(temperature=0.2),  # keep close to lab defaults
            bidding_template=bid_tmpl,
        )
    )

# -----------------------------
# Speaker selection (verbose: prints bids each step)
# -----------------------------
def select_next_speaker_verbose(step: int, agents):
    bids = [ask_for_bid(agent) for agent in agents]
    bid_map = {agent.name: bid for agent, bid in zip(agents, bids)}
    print(f"\n[Step {step}] bids => {bid_map}")

    max_value = np.max(bids)
    max_indices = np.where(bids == max_value)[0]
    return int(np.random.choice(max_indices))

# -----------------------------
#  Run the simulation
# -----------------------------
max_iters = 8
speaker_counts = Counter()

sim = DialogueSimulator(agents=exercise_agents, selection_function=select_next_speaker_verbose)
sim.reset()
sim.inject("Moderator", exercise_topic)
print(f"(Moderator): {exercise_topic}\n")

for _ in range(max_iters):
    speaker, message = sim.step()
    speaker_counts[speaker] += 1
    print(f"({speaker}): {message}\n")

print("=== Speaker counts ===")
for name, count in speaker_counts.most_common():
    print(f"{name}: {count}")


(Moderator): Team: design a Pride & Prejudice Q&A chatbot using a RAG approach. We need: ingestion plan, vector store setup, prompt templates, and an evaluation plan with demo questions.


[Step 1] bids => {'Ingestion Engineer': 10, 'Retrieval Engineer': 10, 'Evaluation Engineer': 10}
(Evaluation Engineer): For the evaluation plan of the Pride & Prejudice Q&A chatbot using a RAG approach, we can consider the following components:

1. **Test Sets**: 
   - Create a diverse set of test questions covering various aspects of Pride & Prejudice, including character names, plot details, themes, and quotes.
   - Include questions that require different levels of understanding, from simple factual queries to more complex inference-based questions.

2. **Metrics**:
   - Use standard metrics such as accuracy, precision, recall, and F1 score to evaluate the performance of the chatbot in providing correct answers.
   - Consider incorporating conversational metrics like engagement, coherence, and rel

## Exercise recap: Relevance-based bidding multi-agent loop (what we built + why it matters)

### What we did
- We reused the lab’s **multi-agent simulator** (agents + simulator + bid parser + speaker selection).
- We added an **Exercise Experiments** cell that changes only the *exercise layer*:
  - A new scenario: *“Build a Pride & Prejudice RAG Q&A chatbot.”*
  - Three role agents:
    - **Ingestion Engineer** (data sourcing, cleaning, chunking, metadata)
    - **Retrieval Engineer** (embeddings, FAISS, retrieval strategy, grounding)
    - **Evaluation Engineer** (metrics, demo questions, failure modes)
  - A new bidding rule: agents bid based on **relevance/importance to respond**, not contradiction.

### The core mechanism
- Each step:
  1. Every agent generates a **bid** (1–10) using a strict format (e.g., `<7>`).
  2. A parser extracts the integer.
  3. The simulator selects the **highest bidder** as the next speaker (ties break randomly).
  4. That speaker generates the next message; everyone receives it and updates memory.

### Why this matters (key takeaway)
This is a clean pattern for building multi-agent systems where:
- You **don’t hardcode turn order**
- You get **emergent coordination** based on what each agent thinks is most useful next
- You can steer behavior by modifying only:
  - the **bidding rubric** (who speaks)
  - the **agent roles/system prompts** (what they say)

---

## What we observed in your run (important signals)
### 1) Bidding created meaningful speaker control
- Step 1: all agents bid **10** (expected right after a fresh objective).
- Step 2: **Ingestion Engineer = 9**, others **0** → only ingestion spoke next.
- This shows the bidding rule can **shift control** to the most relevant role.

### 2) Role prompts improved “division of labor”
- Evaluation agent produced an evaluation plan (metrics, failure modes, demo design).
- Ingestion agent produced concrete ingestion steps (source, cleaning, chunking, metadata).
- The system naturally started building your **project deliverables**.

### 3) Weak spot: repetition / imbalance
- Ingestion repeated itself across steps.
- Retrieval Engineer stayed quiet (0 bids), meaning:
  - the conversation didn’t explicitly request retrieval config, or
  - the bidding rubric didn’t reward “uncovered deliverables”.

---

## How to reuse this later in your final project (copy-paste mental model)

### A) Keep the architecture the same
Use the same pattern:
- **Agents (roles)**
- **Bid → parse → select next speaker**
- **Simulator step loop**
- **Injected moderator objective**

### B) Tune these 3 levers (most important)
1) **Roles** (SystemMessage / headers)
   - Make each agent own a deliverable.
   - Example roles for RAG:
     - Data/Ingestion
     - Retrieval/Indexing
     - Prompting/Guardrails
     - Evaluation/Testing
     - Product/UX (optional)

2) **Bidding rubric** (controls “who speaks next”)
   - Use a rubric aligned to *progress toward deliverables*.
   - Good rubric factors:
     - “Is this in my domain?”
     - “Am I adding something new?”
     - “Is a deliverable missing?”
     - “Is correction urgently needed?”

3) **Format enforcement + robustness**
   - Always require bid output in a strict format (`<int>`).
   - Keep parsing + retry + fallback to avoid chain breaks.

---

## Quick improvements you can apply next time (small changes, big impact)

### 1) Prevent repetition
Add to bidding rules:
- “If you already covered this topic in the last 1–2 turns, bid `<1>`.”

### 2) Ensure all deliverables get covered
Add to bidding rules:
- “If the conversation has not produced *your* deliverable yet, bid higher.”

Example:
- Retrieval agent bids high until it has produced:
  - embedding model choice
  - FAISS index type
  - retrieval top_k
  - metadata filtering plan

### 3) Force structured artifact outputs
In the moderator prompt (or agent system prompts), require headings:
- `### Ingestion Plan`
- `### Vector Store & Retrieval`
- `### Prompt Templates`
- `### Evaluation + Demo Questions`

This makes the conversation output directly usable in a README or report.

---

## One-line summary
We used the lab’s multi-agent bidding simulator and changed the bidding rubric to **relevance-based**, producing a practical, role-specialized planning conversation for a RAG chatbot—exactly the kind of coordination pattern you can reuse in your final project.